In [1]:
from davitpy.models import *
from davitpy.pydarn.plotting import *
import davitpy.gme as gme
import datetime
import pandas
import numpy
from davitpy import utils
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.ticker import LinearLocator
import seaborn as sns
from pymongo import MongoClient
import sys
%matplotlib inline

In [2]:
plotCoords = "mag"
selTime = datetime.datetime( 2015, 1, 1, 3, 0 )
selTimeDstHigh = datetime.datetime( 2015, 1, 1, 3, 0 )
selTimeDst57 = datetime.datetime( 2015, 1, 1, 3, 30 )
selTimeDstLow = datetime.datetime( 2015, 1, 1, 4, 0 )
selTimeDst18 = datetime.datetime( 2015, 1, 1, 4, 30 )
selTimeDst5 = datetime.datetime( 2015, 1, 1, 5, 0 )
sapsProbsFName = "../data/sapsProb-Models.txt"
# coords of collges
geoLatDartMouth = 43.7
geoLonDartMouth = -72.3
geoLatUMich = 42.27
geoLonUMich = -83.73
geoLatUWM = 43.07
geoLonUWM = -89.41
geoLatUMn = 44.97
geoLonUMn = -93.22

In [3]:
# Need a function to convert MLT to MLON
def convert_mlt_to_mlon(inpMLT, selTime=selTime):
    outMlon = None
    prevDiff = None
    for currMlon in range(0, 360):
        currMLT = aacgm.mltFromYmdhms(\
                          selTime.year, selTime.month,
                                    selTime.day, selTime.hour,
                                    selTime.minute, selTime.second,
                                    currMlon) 
        if abs(currMLT-inpMLT) < 0.1:
            if outMlon is None:
                outMlon = currMlon
                prevDiff = abs(currMLT-inpMLT)
            else:
                if prevDiff > abs(currMLT-inpMLT):
                    outMlon = currMlon
                    prevDiff = abs(currMLT-inpMLT)                
    return outMlon

def saps_pred_func(x, y, dst):    
    (a_sx, b_sx, a_sy, b_sy, a_xo, b_xo, a_yo, b_yo, a_o, b_o, theta) = \
        ( 3.04, 2.98e-03, 1.69,\
         -8.53e-04, 4.61, 6.36e-02, -1.28, 3.1e-02,
         8.49e-01, -1.57e-03, 6.69e-01)        
    sigma_x = a_sx + b_sx * dst
    sigma_y = a_sy + b_sy * dst
    xo = a_xo + b_xo * dst
    yo = a_yo + b_yo * dst
    amplitude = a_o + b_o * dst    
    
    a = (numpy.cos(theta)**2)/(2*sigma_x**2) + (numpy.sin(theta)**2)/(2*sigma_y**2)
    b = -(numpy.sin(2*theta))/(4*sigma_x**2) + (numpy.sin(2*theta))/(4*sigma_y**2)
    c = (numpy.sin(theta)**2)/(2*sigma_x**2) + (numpy.cos(theta)**2)/(2*sigma_y**2)
    g = amplitude*numpy.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g

In [4]:
# read SAPS model data
sapsModelDF = pandas.read_csv(sapsProbsFName, sep=' ')
sapsModelDF["pred_Mlon"] = [ convert_mlt_to_mlon(x) for x in sapsModelDF["pred_MLT"] ]
sapsModelDF.head()
sapsModelDF = sapsModelDF[ sapsModelDF["pred_prob"] >= 0.2 ].reset_index(drop=True)
# sapsModelDF = sapsModelDF[ sapsModelDF["dst_median"] == -95. ]
# print sapsModelDF["dst_median"].unique()
# latCntr = sapsModelDF["pred_Lat"].reshape( ( 5, 10 ) )
# lonCntr = sapsModelDF["pred_Mlon"].reshape( ( 5, 10 ) )
# probCntr = sapsModelDF["pred_prob"].reshape( ( 5, 10 ) )
latCntr, lonCntr = numpy.meshgrid( sapsModelDF["pred_Lat"],\
                                            sapsModelDF["pred_Mlon"] )
probCntr = numpy.zeros( latCntr.shape )
for i1 in range( probCntr.shape[0] ):
    for i2 in range( probCntr.shape[1] ):
        currProbList = sapsModelDF[ (sapsModelDF["pred_Lat"] == latCntr[i1][i2]) & \
                                       (sapsModelDF["pred_Mlon"] == lonCntr[i1][i2]) ]["pred_prob"].tolist()
        if len(currProbList) > 0:
            probCntr[i1][i2] = currProbList[0]
        else:
            probCntr[i1][i2] = numpy.nan

In [ ]:
sapsPredDF = pandas.DataFrame(columns=["normMLT", "normLAT", "probSAPS", "MLT", "Lat", "dst_index"])
latArr = []
mltArr = []
normLatArr = []
normMltArr = []
probSapsArr = []
dstValArr = []
dstBinArr = []

dstValsInpList = [ -25., -120. ]

for z in range( len( dstValsInpList ) ):
    for x in range( 45, 70 ):
        for y in range( 0, 360 ):        
            normLatArr.append( x - 57.5 )
            mltArr.append( y )
            if y < 12:
                currNormMLT = y
            else:
                currNormMLT = y - 24.
            normMltArr.append( currNormMLT )
            latArr.append( x  )
            currDstVal = dstValsInpList[z]
            currProbVal = saps_pred_func(x-57.5, currNormMLT, currDstVal)
            if currProbVal > 0.001:
                probSapsArr.append( currProbVal )
            else:
                probSapsArr.append( 0. )
            dstValArr.append(currDstVal)
        
sapsPredDF["pred_MLT"] = mltArr
sapsPredDF["pred_Lat"] = latArr
sapsPredDF["normMLT"] = normMltArr
sapsPredDF["normLAT"] = normLatArr
sapsPredDF["probSAPS"] = probSapsArr
sapsPredDF["dst_index"] = dstValArr
# Dst High
sapsPredDFDstHigh = sapsPredDF[ sapsPredDF["dst_index"] == dstValsInpList[1] ]
sapsPredDFDstHigh["pred_Mlon"] = [ convert_mlt_to_mlon(x, selTime=selTimeDstHigh)\
                                for x in sapsPredDFDstHigh["pred_MLT"] ]
latCntrDstHigh, lonCntrDstHigh = numpy.meshgrid( sapsPredDFDstHigh["pred_Lat"],\
                                            sapsPredDFDstHigh["pred_Mlon"] )
probCntrDstHigh = numpy.zeros( latCntrDstHigh.shape )
for i1 in range( probCntr.shape[0] ):
    for i2 in range( probCntr.shape[1] ):
        currProbList = sapsPredDFDstHigh[ (sapsPredDFDstHigh["pred_Lat"] == latCntrDstHigh[i1][i2]) & \
                                       (sapsPredDFDstHigh["pred_Mlon"] == lonCntrDstHigh[i1][i2]) ]["probSAPS"].tolist()
        if len(currProbList) > 0:
            probCntrDstHigh[i1][i2] = currProbList[0]
        else:
            probCntrDstHigh[i1][i2] = numpy.nan#None#numpy.nan
#             latCntrDstHigh[i1][i2] = numpy.nan#None
#             lonCntrDstHigh[i1][i2] = numpy.nan#None
# # Dst -57 nT
# sapsPredDFDst57 = sapsPredDF[ sapsPredDF["dst_index"] == -57. ]
# sapsPredDFDst57["pred_Mlon"] = [ convert_mlt_to_mlon(x, selTime=selTimeDst57)\
#                                 for x in sapsPredDFDst57["pred_MLT"] ]
# latCntrDst57, lonCntrDst57 = numpy.meshgrid( sapsPredDFDst57["pred_Lat"],\
#                                             sapsPredDFDst57["pred_Mlon"] )
# probCntrDst57 = numpy.zeros( latCntrDst57.shape )
# for i1 in range( probCntrDst57.shape[0] ):
#     for i2 in range( probCntrDst57.shape[1] ):
#         currProbList = sapsPredDFDst57[ (sapsPredDFDst57["pred_Lat"] == latCntrDst57[i1][i2]) & \
#                                        (sapsPredDFDst57["pred_Mlon"] == lonCntrDst57[i1][i2]) ]["probSAPS"].tolist()
#         if len(currProbList) > 0:
#             probCntrDst57[i1][i2] = currProbList[0]
#         else:
#             probCntrDst57[i1][i2] = numpy.nan#None#numpy.nan
#             latCntrDst57[i1][i2] = numpy.nan#None
#             lonCntrDst57[i1][i2] = numpy.nan#None
# Dst Low
sapsPredDFDstLow = sapsPredDF[ sapsPredDF["dst_index"] == dstValsInpList[0] ]
sapsPredDFDstLow["pred_Mlon"] = [ convert_mlt_to_mlon(x, selTime=selTimeDstLow)\
                                for x in sapsPredDFDstLow["pred_MLT"] ]
latCntrDstLow, lonCntrDstLow = numpy.meshgrid( sapsPredDFDstLow["pred_Lat"],\
                                            sapsPredDFDstLow["pred_Mlon"] )
probCntrDstLow = numpy.zeros( latCntrDstLow.shape )
for i1 in range( probCntrDstLow.shape[0] ):
    for i2 in range( probCntrDstLow.shape[1] ):
        currProbList = sapsPredDFDstLow[ (sapsPredDFDstLow["pred_Lat"] == latCntrDstLow[i1][i2]) & \
                                       (sapsPredDFDstLow["pred_Mlon"] == lonCntrDstLow[i1][i2]) ]["probSAPS"].tolist()
        if len(currProbList) > 0:
            probCntrDstLow[i1][i2] = currProbList[0]
        else:
            probCntrDstLow[i1][i2] = numpy.nan#None#numpy.nan
#             latCntrDstLow[i1][i2] = numpy.nan#None
#             lonCntrDstLow[i1][i2] = numpy.nan#None
# # Dst -18 nT
# sapsPredDFDst18 = sapsPredDF[ sapsPredDF["dst_index"] == -18. ]
# sapsPredDFDst18["pred_Mlon"] = [ convert_mlt_to_mlon(x, selTime=selTimeDst18)\
#                                 for x in sapsPredDFDst18["pred_MLT"] ]
# latCntrDst18, lonCntrDst18 = numpy.meshgrid( sapsPredDFDst18["pred_Lat"],\
#                                             sapsPredDFDst18["pred_Mlon"] )
# probCntrDst18 = numpy.zeros( latCntrDst18.shape )
# for i1 in range( probCntrDst18.shape[0] ):
#     for i2 in range( probCntrDst18.shape[1] ):
#         currProbList = sapsPredDFDst18[ (sapsPredDFDst18["pred_Lat"] == latCntrDst18[i1][i2]) & \
#                                        (sapsPredDFDst18["pred_Mlon"] == lonCntrDst18[i1][i2]) ]["probSAPS"].tolist()
#         if len(currProbList) > 0:
#             probCntrDst18[i1][i2] = currProbList[0]
#         else:
#             probCntrDst18[i1][i2] = numpy.nan#None#numpy.nan
#             latCntrDst18[i1][i2] = numpy.nan#None
#             lonCntrDst18[i1][i2] = numpy.nan#None
# # Dst -5 nT
# sapsPredDFDst5 = sapsPredDF[ sapsPredDF["dst_index"] == -5. ]
# sapsPredDFDst5["pred_Mlon"] = [ convert_mlt_to_mlon(x, selTime=selTimeDst5)\
#                                 for x in sapsPredDFDst5["pred_MLT"] ]
# latCntrDst5, lonCntrDst5 = numpy.meshgrid( sapsPredDFDst5["pred_Lat"],\
#                                             sapsPredDFDst5["pred_Mlon"] )
# probCntrDst5 = numpy.zeros( latCntrDst5.shape )
# for i1 in range( probCntrDst5.shape[0] ):
#     for i2 in range( probCntrDst5.shape[1] ):
#         currProbList = sapsPredDFDst5[ (sapsPredDFDst5["pred_Lat"] == latCntrDst5[i1][i2]) & \
#                                        (sapsPredDFDst5["pred_Mlon"] == lonCntrDst5[i1][i2]) ]["probSAPS"].tolist()
#         if len(currProbList) > 0:
#             probCntrDst5[i1][i2] = currProbList[0]
#         else:
#             probCntrDst5[i1][i2] = 0.01#numpy.nan#None#numpy.nan
# #             latCntrDst5[i1][i2] = numpy.nan#None
# #             lonCntrDst5[i1][i2] = numpy.nan#None

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# READ MHO coords from DB and convert them to MAGN coords
SDDB = "sd-work9.ece.vt.edu:27017"
SDBREADUSER = "sd_dbread"
SDBREADPASS = "5d"
conn = MongoClient('mongodb://{}:{}@{}/{}'.format(SDBREADUSER,
                                                              SDBREADPASS,
                                                              SDDB,
                                                              "isr"))

dbConn = conn["isr"]
qIn = {'code': "mho"}
qRes = dbConn.info.find(qIn)
for el in qRes:
    magPosMHOLon, magPosMHOLat = utils.coord_conv( el['pos']['lon'], el['pos']['lat'], \
                                 "geo", "mag", altitude=300., \
                                 date_time=selTime )
    magFovMHOLon, magFovMHOLat = utils.coord_conv( el['fov']['lon'], el['fov']['lat'], \
                                 "geo", "mag", altitude=300., \
                                 date_time=selTime )

In [ ]:
f = plt.figure(figsize=(12, 8))
ax = f.add_subplot(1,1,1)
# set colorbar
seaMap = ListedColormap(sns.color_palette("Reds"))
# m1 = utils.plotUtils.mapObj(boundinglat=40., gridLabels=True,\
#                             coords=plotCoords, ax=ax, datetime=selTime)
m1 = utils.plotUtils.mapObj(lat_0=70., lon_0=-40, width=111e3*120,\
                      height=111e3*55, coords='mag', ax=ax, datetime=selTime)

codes = ['wal','fhe','fhw','cve','cvw','ade','adw','bks']
# Plotting some radars
overlayRadar(m1, fontSize=12, codes=codes)
# Plot radar fov
overlayFov(m1, codes=codes, maxGate=70)
# overlay MHO ISR
xPosMHO, yPosMHO = m1( magPosMHOLon, magPosMHOLat )
m1.scatter(xPosMHO, yPosMHO, zorder=6, s=20, facecolors='k')
m1.ax.text(xPosMHO * 1.02, yPosMHO * 0.98, "MHO", zorder=6)
xFovMHO, yFovMHO = m1( magFovMHOLon, magFovMHOLat )
m1.plot(xFovMHO, yFovMHO, 'k')

# xVec, yVec = m1(list(sapsPredDF["pred_Mlon"]), list(sapsPredDF["pred_Lat"]), coords=plotCoords)
# modPlot = m1.scatter( xVec, yVec , c=sapsPredDF["probSAPS"], s=40.,\
#            cmap=seaMap, alpha=0.7, zorder=5., \
#                      edgecolor='none', marker="." )

xCntrDstHigh, yCntrDstHigh = m1(lonCntrDstHigh, latCntrDstHigh, coords=plotCoords)
cntrPltDstHigh = m1.contour( xCntrDstHigh, yCntrDstHigh, probCntrDstHigh, 
            zorder = 2., levels=[0.25],
            colors = 'firebrick', linewidths=1.,linestyles='dashed' )

# xCntrDst57, yCntrDst57 = m1(lonCntrDst57, latCntrDst57, coords=plotCoords)
# cntrPltDst57 = m1.contour( xCntrDst57, yCntrDst57, probCntrDst57, 
#             zorder = 2., levels=[0.25],
#             colors = 'orangered', linewidths=1. )

xCntrDstLow, yCntrDstLow = m1(lonCntrDstLow, latCntrDstLow, coords=plotCoords)
cntrPltDstLow = m1.contour( xCntrDstLow, yCntrDstLow, probCntrDstLow, 
            zorder = 2., levels=[0.25],
            colors = 'seagreen', linewidths=1.,linestyles='dashed' )

# xCntrDst18, yCntrDst18 = m1(lonCntrDst18, latCntrDst18, coords=plotCoords)
# cntrPltDst18 = m1.contour( xCntrDst18, yCntrDst18, probCntrDst18,
#             zorder = 2., levels=[0.25],
#             colors = 'slateblue', linewidths=1. )

# xCntrDst5, yCntrDst5 = m1(lonCntrDst5, latCntrDst5, coords=plotCoords)
# cntrPltDst5 = m1.contour( xCntrDst5, yCntrDst5, probCntrDst5,
#             zorder = 2.,
#             colors = 'dodgerblue', linewidths=1. )


# cbar = plt.colorbar(modPlot, orientation='vertical')
# cbar.set_label('TEC', size=15)
# plt.clabel(cntrPltDstHigh, inline=1, fontsize=10)
# plt.clabel(cntrPltDst57, inline=1, fontsize=10)
# plt.clabel(cntrPltDstLow, inline=1, fontsize=10)
# plt.clabel(cntrPltDst18, inline=1, fontsize=10)
# plt.clabel(cntrPltDst5, inline=1, fontsize=10)
f.savefig("../figs/map-saps-gps.pdf",bbox_inches='tight')